In [2]:
from sklearn.datasets import  load_iris
dataset=load_iris()
#获得ndarray
x=dataset.data
#获得样本类别
#print(x)
y=dataset.target
#print(y)

#print(dataset.DESCR)

#### 求得各个特征值的平均值

In [3]:
attribute_means=x.mean(axis=0)
print(attribute_means)

[ 5.84333333  3.054       3.75866667  1.19866667]


#### 离散化，将大于平均值的设置为1，小于平均值的设置为0

In [4]:
import numpy as np
x_d=np.array(x>=attribute_means,dtype=int)
print(x_d[:5])

[[0 1 0 0]
 [0 0 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 1 0 0]]


## <a src="http://blog.csdn.net/cherdw/article/details/54881167">参考</a>

In [13]:
from sklearn.cross_validation import  train_test_split
random_state=14
x_train,x_test,y_train,y_test=train_test_split(x_d,y,random_state=random_state)

print("总共有{}个训练例子".format(y_train.shape))
print("总共有{}个测试例子".format(y_test.shape))

总共有(112,)个训练例子
总共有(38,)个测试例子


In [17]:
from collections import defaultdict
from operator import itemgetter
def train(X, y_true, feature):
    
    # Check that variable is a valid 
    n_samples, n_features = X.shape
    assert 0 <= feature < n_features
    # Get all of the unique values that this variable has
    values = set(X[:,feature])
    # Stores the predictors array that is returned
    predictors = dict()
    errors = []
    for current_value in values:
        most_frequent_class, error = train_feature_value(X, y_true, feature, current_value)
        predictors[current_value] = most_frequent_class
        errors.append(error)
    # Compute the total error of using this feature to classify on
    total_error = sum(errors)
    return predictors, total_error

# Compute what our predictors say each sample is based on its value
#y_predicted = np.array([predictors[sample[feature]] for sample in X])
    

def train_feature_value(X, y_true, feature, value):
    # Create a simple dictionary to count how frequency they give certain predictions
    class_counts = defaultdict(int)
    # Iterate through each sample and count the frequency of each class/value pair
    for sample, y in zip(X, y_true):
        if sample[feature] == value:
            class_counts[y] += 1
    # Now get the best one by sorting (highest first) and choosing the first item
    sorted_class_counts = sorted(class_counts.items(), key=itemgetter(1), reverse=True)
    most_frequent_class = sorted_class_counts[0][0]
    # The error is the number of samples that do not classify as the most frequent class
    # *and* have the feature value.
    n_samples = X.shape[1]
    error = sum([class_count for class_value, class_count in class_counts.items()
                 if class_value != most_frequent_class])
    return most_frequent_class, error

In [19]:
all_predictors={variable: train(x_train,y_train,variable) for variable in range(x_train.shape[1])}
errors = {variable: error for variable, (mapping, error) in all_predictors.items()}
# Now choose the best and save that as "model"
# Sort by error
best_variable, best_error = sorted(errors.items(), key=itemgetter(1))[0]
print("The best model is based on variable {0} and has error {1:.2f}".format(best_variable, best_error))

# Choose the bset model
model = {'variable': best_variable,
         'predictor': all_predictors[best_variable][0]}
print(model)

The best model is based on variable 2 and has error 37.00
{'variable': 2, 'predictor': {0: 0, 1: 2}}
